In [22]:
import pandas as pd
import os, time
from multiprocessing import Pool
import numpy as np
import glob
import seaborn as sns

In [23]:
parent_path ='C:\PRABHANJAN\EdNET_100Users\\'
fnames=glob.glob(parent_path+"/*.csv")
user_list=[]
for filename in fnames:
    df=pd.read_csv(filename)
    filename=filename.replace(parent_path,'')
    filename=filename.replace('.csv','')
    df['user_id']=filename
    user_list.append(df)
    

userRef_df= pd.concat(user_list,axis=0)


In [24]:
del userRef_df['timestamp']
del userRef_df['action_type']
del userRef_df['cursor_time']
del userRef_df['source']
del userRef_df['platform']

userRef_df

,item_id,user_answer,user_id
0,b3544,NaN,u1
1,q5012,b,u1
2,b3544,NaN,u1
3,b3238,NaN,u1
4,q4706,c,u1
...,...,...,...
3955,b447,NaN,u999
3956,e447,NaN,u999
3957,b447,NaN,u999
3958,b447,NaN,u999


In [25]:
userRef_df.dropna()  #removed rows without user's question attempt

,item_id,user_answer,user_id
1,q5012,b,u1
4,q4706,c,u1
7,q4366,b,u1
10,q4829,a,u1
13,q6528,b,u1
...,...,...,...
3941,q997,a,u999
3943,q997,c,u999
3944,q997,c,u999
3952,q447,a,u999


In [27]:

questions_file_path = "C:\\Users\\Prabhanjan Kulkarni\\Desktop\\IS 733\\contents\\questions.csv"
questions_df = pd.read_csv(questions_file_path)

del questions_df['bundle_id']
del questions_df['explanation_id']
del questions_df['part']
del questions_df['tags']
del questions_df['deployed_at']


In [28]:
userfinal_df=pd.merge(userRef_df, questions_df, left_on='item_id', right_on='question_id')
userfinal_df["final"]=np.where(userfinal_df['user_answer']==userfinal_df['correct_answer'],True,False)
userfinal_df

,item_id,user_answer,user_id,question_id,correct_answer,final
0,q5012,b,u1,q5012,c,False
1,q5012,b,u1014,q5012,c,False
2,q5012,c,u1014,q5012,c,True
3,q5012,c,u1014,q5012,c,True
4,q5012,a,u1043,q5012,c,False
...,...,...,...,...,...,...
1194763,q18056,a,u775,q18056,c,False
1194764,q18056,c,u929,q18056,c,True
1194765,q18056,a,u948,q18056,c,False
1194766,q18056,b,u948,q18056,c,False


In [29]:
userprofile = userfinal_df['user_id'].value_counts()


In [30]:
userprofile_table=pd.DataFrame({'user_id':userprofile.index, 'attempts':userprofile.values})

In [31]:
userprofile_table

,user_id,attempts
0,u226,41348
1,u995,32771
2,u622,19709
3,u1014,18695
4,u743,18088
...,...,...
994,u943,5
995,u1069,3
996,u517,2
997,u291,2


In [32]:
userName = userprofile_table['user_id'].unique()
correct=[]
j=0
for i in range(len(userName)):
    counter = 0
    for index,row in userfinal_df.iterrows():
        if userName[i] == row['user_id'] and row['final']==True:
                counter+=1
    correct.append(counter)

            
        

In [33]:
userprofile_table['correct_answers']=correct

In [34]:
userprofile_table['user_accuracy']=userprofile_table['correct_answers'].astype(float)/userprofile_table['attempts'].astype(float)

In [35]:
lec_wat=0
exp_read=0
user_lec=[]
user_exp=[]
user_index=[]
def get_count_of_lec_explanation(files,a):
    global lec_wat
    global exp_read
    global user_lec
    global user_exp
    global user_index
    current_df = pd.read_csv(files)
    count_lec = 0
    count_exp = 0
    for index, row in current_df.iterrows():
        item_id = row["item_id"]
        if item_id[:1] == "l":
            # print(file[1])
            #print(item_id[:1])
            count_lec += 1
        if item_id[:1] == "e":
            count_exp += 1
    user_index.append(a)
    user_lec.append(count_lec)
    user_exp.append(count_exp)
    lec_wat+=count_lec
    exp_read+=count_exp
    print("For user {0} Lecturs watched={1} and Explaination Read={2}\n".format(a,count_lec,count_exp))

In [36]:
parent_path = 'C:\PRABHANJAN\EdNET_100Users\\'
path = glob.glob(parent_path+"/*.csv")
for filename in path:
        user_id=filename
        user_id=user_id.replace(parent_path,'')
        user_id=user_id.replace('.csv','')
        get_count_of_lec_explanation(filename,user_id)
#c1, c2 = get_count_of_lec_explanation(file_names[0])
print("Total Lectures watched=",lec_wat)
print("Total explaination read=",exp_read)

insight_df=pd.DataFrame()
insight_df['user_id']=user_index
insight_df['Lecture_Watched']=user_lec
insight_df['Explaination_Read']=user_exp

For user u1 Lecturs watched=124 and Explaination Read=1100

For user u10 Lecturs watched=0 and Explaination Read=2

For user u100 Lecturs watched=8 and Explaination Read=28

For user u1000 Lecturs watched=1029 and Explaination Read=2038

For user u1001 Lecturs watched=0 and Explaination Read=0

For user u1002 Lecturs watched=30 and Explaination Read=194

For user u1003 Lecturs watched=32 and Explaination Read=274

For user u1005 Lecturs watched=2 and Explaination Read=136

For user u1006 Lecturs watched=8 and Explaination Read=6

For user u1007 Lecturs watched=12 and Explaination Read=712

For user u1008 Lecturs watched=16 and Explaination Read=208

For user u1009 Lecturs watched=28 and Explaination Read=212

For user u101 Lecturs watched=4 and Explaination Read=0

For user u1010 Lecturs watched=10 and Explaination Read=10

For user u1011 Lecturs watched=60 and Explaination Read=584

For user u1012 Lecturs watched=1403 and Explaination Read=4048

For user u1013 Lecturs watched=8 and Ex

In [40]:
insight_df

,user_id,Lecture_Watched,Explaination_Read
0,u1,124,1100
1,u10,0,2
2,u100,8,28
3,u1000,1029,2038
4,u1001,0,0
...,...,...,...
996,u995,1914,16112
997,u996,34,8
998,u997,13,194
999,u998,194,242


In [41]:
Final_UserProfileTable=pd.merge(userprofile_table, insight_df, left_on='user_id', right_on='user_id')
Final_UserProfileTable

,user_id,attempts,correct_answers,user_accuracy,Lecture_Watched,Explaination_Read
0,u226,41348,12339,0.298418,440,6616
1,u995,32771,11845,0.361448,1914,16112
2,u622,19709,7976,0.404688,343,4900
3,u1014,18695,8982,0.480449,685,18094
4,u743,18088,8216,0.454224,3075,19876
...,...,...,...,...,...,...
994,u943,5,2,0.400000,0,8
995,u1069,3,1,0.333333,6,2
996,u517,2,2,1.000000,4,4
997,u291,2,1,0.500000,0,2


In [43]:
Final_UserProfileTable.to_csv('UserProfile_Table_Q1.csv')